<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩 (https://school.fun-coding.org/)</a> 에서 본 강의를 포함하는 최적화된 로드맵도 확인하실 수 있습니다</b></div>

In [ ]:
import torch
import torch.nn as nn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from copy import deepcopy

In [ ]:
df = pd.read_csv('dataset/AMZN.csv', index_col = 'Date', parse_dates=True)
df.head(5)

### MinMaxScaler 적용

In [ ]:
from sklearn.preprocessing import MinMaxScaler

raw_data = deepcopy(df.drop(['Name'], axis=1))
X_scaler = MinMaxScaler(feature_range=(-1, 1))
y_scaler = MinMaxScaler(feature_range=(-1, 1))
X_raw_data = X_scaler.fit_transform(raw_data)
y_raw_data = y_scaler.fit_transform(raw_data['Close'].values.reshape(-1, 1))
print (X_raw_data.shape, y_raw_data.shape)

### X_train 과 y_train 데이터 구성

In [ ]:
def prepare_xy(X_raw_data, y_raw_data, lookback):
    data = list()
    for index in range(len(X_raw_data) - lookback): 
        data.append(X_raw_data[index: index + lookback])
    data = np.array(data)
    return data, y_raw_data[lookback:]

In [ ]:
lookback = 10 # sequence length - 1
X_train, y_train = prepare_xy(X_raw_data, y_raw_data, lookback)
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
print(X_train.shape, y_train.shape)

### Custom Dataset
- MNIST 데이터는 이미 잘 만들어진 데이터로, 바로 Subset, DataLoader 등 파이토치의 미니배치 구성 관련 기능을 사용할 수 있음
- raw data 를 기반으로, 파이토치의 미니배치등을 구성하려면, 위와 같이 직접 데이터셋을 작성하여, TensorDataset 을 통해, Dataset 으로 만들면 됨
- 이 기법 외에 Custom Dataset 을 구성할 수도 있음

> raw data 를 텐서로 만들어, 전체 데이터를 딥러닝 모델에 한번에 넣는 경우는 없으므로, 미니배치를 쉽게 구성하고, 반복문을 통해, 미니배치 하나씩 가져오도록 하는 구성이 필요함
> 이를 손쉽게 해주는 기능이 Subset, DataLoader 등의 기능이며, 이를 위해, raw data 를 Dataset 으로 만들어주어야 함

### Custom Dataset class
- Custom Dataset 은 클래스로 구현해야 함 (모델 구현과 유사함)
- torch.utils.data.Dataset 클래스를 상속받아야 함
- 다음 세가지 메서드가 필요함
  - \_\_init\_\_(self) : 입력 데이터(x)와 실제 값(y) 을 선언해주는 메서드
  - \_\_len\_\_(self) :입력 데이터(x)와 실제 값(y) 길이를 리턴해주는 메서드
  - \_\_getitem\_\_(self, index) : index번째 입력 데이터(x)와 실제 값(y) 을 리턴해주는 메서드

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.X_data = X_train
        self.y_data = y_train

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.X_data)

    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, index): 
        X = self.X_data[index]
        y = self.y_data[index]
        return X, y

In [ ]:
train_rawdata = CustomDataset()
train_rawdata.y_data

### TensorDataset
- TensorDataset은 Dataset을 상속한 클래스로 학습 데이터 X와 실제 값 Y를 묶어 놓는 Dataset
   - 학습 데이터와 실제 값을 하나로 묶어서, 인덱스, 반복문을 통한 각 데이터 추출을 편리하게 하는 기능
   - DataLoader 등 pytorch 의 데이터 전처리 기능 사용 가능
- TensorDataset으로 랩핑한 Dataset 은 DataLoader 로 미니배치를 쉽게 작성할 수 있음
- tensors() 메서드로 각 텐서를 인덱스 번호로 엑세스도 가능함

In [ ]:
train_rawdata = TensorDataset(X_train, y_train)

In [ ]:
print (train_rawdata.tensors[0].shape) # X_train
print (train_rawdata.tensors[1].shape) # y_train

### Generate Train & Validation Mini-batch
- TensorDataset 으로 만든 Dataset 이든, Custom Dataset class 로 만든 Dataset 이든 모두 사용 가능

In [ ]:
VALIDATION_RATE = 0.2
train_indices, val_indices = train_test_split(
    range(len(train_rawdata)), # X index 번호
    test_size=VALIDATION_RATE # test dataset 비율
)
train_dataset = Subset(train_rawdata, train_indices)
validation_dataset = Subset(train_rawdata, val_indices)

minibatch_size = 128 # Mini-batch 사이즈는 128 로 설정
# create batches
train_batches = DataLoader(train_dataset, batch_size=minibatch_size, shuffle=True)
val_batches = DataLoader(validation_dataset, batch_size=minibatch_size, shuffle=True)

In [ ]:
# 미니배치 하나만 가져와서 이미지 visualization 
X_train, y_train = next(iter(train_batches))
print (X_train.shape, y_train.shape)

### 모델 생성

In [ ]:
class Net(nn.Module):
    def __init__(self, feature_size, hidden_size, num_layers, dropout_p, output_size, model_type='LSTM'):
        super().__init__()
        if model_type == 'LSTM':
            self.sequenceclassifier = nn.LSTM(
                input_size = feature_size,
                hidden_size = hidden_size,
                num_layers = num_layers,
                batch_first = True,
                dropout = dropout_p
            )
        elif model_type == 'GRU':
            self.sequenceclassifier = nn.GRU(
                input_size = feature_size,
                hidden_size = hidden_size,
                num_layers = num_layers,
                batch_first = True,
                dropout = dropout_p
            )
        self.fc = nn.Sequential(
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(hidden_size),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        output, _ = self.sequenceclassifier(x) # |output| = (128, 10, 32)
        output = output[:, -1, :] # |output| = (128, 32)
        y = self.fc(output)  
        return y 

### input, output, loss, optimizer 설정

In [ ]:
feature_size = 5 # 입력 차원
hidden_size = 32 # Hidden Layer 사이즈 설정처럼 설정
num_layers = 2 # stacked RNN (최대 4개까지는 Gradient Vanishing 현상이 적을 수 있으므로)
dropout_p = 0 # dropout rate
output_size = 1

model = Net(feature_size, hidden_size, num_layers, dropout_p, output_size, 'GRU')
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

### training 함수 선언

In [ ]:
def train_model(model, early_stop, n_epochs, progress_interval):
    
    train_losses, valid_losses, lowest_loss = list(), list(), np.inf

    for epoch in range(n_epochs):
        
        train_loss, valid_loss = 0, 0
        
        # train the model
        model.train() # prep model for training
        for x_minibatch, y_minibatch in train_batches:
            y_minibatch_pred = model(x_minibatch)
            loss = loss_func(y_minibatch_pred, y_minibatch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss = train_loss / len(train_batches)
        train_losses.append(train_loss)      
        
        # validate the model
        model.eval()
        with torch.no_grad():
            for x_minibatch, y_minibatch in val_batches:
                y_minibatch_pred = model(x_minibatch)
                loss = loss_func(y_minibatch_pred, y_minibatch)
                valid_loss += loss.item()
                
        valid_loss = valid_loss / len(val_batches)
        valid_losses.append(valid_loss)

        if valid_losses[-1] < lowest_loss:
            lowest_loss = valid_losses[-1]
            lowest_epoch = epoch
            best_model = deepcopy(model.state_dict())
        else:
            if (early_stop > 0) and lowest_epoch + early_stop < epoch:
                print ("Early Stopped", epoch, "epochs")
                break
                
        if (epoch % progress_interval) == 0:
            print (train_losses[-1], valid_losses[-1], lowest_loss, lowest_epoch, epoch)
            
    model.load_state_dict(best_model)        
    return model, lowest_loss, train_losses, valid_losses

### 훈련 실행
<div class="alert alert-block" style="border: 2px solid #E65100;background-color:#FFF3E0;padding:10px">
<font size="4em" style="color:#BF360C;">본 코드는 CPU 만으로도 테스트하기 때문에, colab 과 GPU 기반으로 실행할 필요는 없습니다. 따라서, 본 코드 기반, 각자 PC 상에서 테스트해보셔도 좋을 것 같습니다</font>
</div>

In [ ]:
nb_epochs = 100 
progress_interval = 3
early_stop = 30

model, lowest_loss, train_losses, valid_losses = train_model(model, early_stop, nb_epochs, progress_interval)

### 테스트
- torch.cat(합칠텐서리스트, 합치는 차원)
   - 합칠텐서 리스트는 기본적으로는 튜플 형태로 넣어야 하지만, 리스트도 가능함

In [ ]:
print (torch.cat([torch.randn(24, 128, 1), torch.randn(24, 128, 1), torch.randn(24, 128, 1)], 0).shape)
print (torch.cat([torch.randn(24, 128, 1), torch.randn(24, 128, 1), torch.randn(24, 128, 1)], 1).shape)
print (torch.cat([torch.randn(24, 128, 1), torch.randn(24, 128, 1), torch.randn(24, 128, 1)], 2).shape)

In [ ]:
test_batches = DataLoader(train_rawdata, batch_size=minibatch_size, shuffle=False)
y_test_pred_list, y_test_list = list(), list()
model.eval()
with torch.no_grad():
    for x_minibatch, y_minibatch in test_batches:
        y_minibatch_pred = model(x_minibatch)
        y_test_pred_list.append(y_minibatch_pred)
        y_test_list.append(y_minibatch)
y_test_preds = torch.cat(y_test_pred_list, 0)
y_tests = torch.cat(y_test_list, 0)
print (y_test_preds.shape, y_tests.shape)

### 데이터 스케일 복원

In [ ]:
predict = pd.DataFrame(y_scaler.inverse_transform(np.array(y_test_preds)))
original = pd.DataFrame(y_scaler.inverse_transform(np.array(y_tests)))

In [ ]:
predict.head()

### Metrics

In [ ]:
from sklearn.metrics import mean_squared_error

RMSE = mean_squared_error(original[0], predict[0])**0.5
print (RMSE)

### 시각화

In [ ]:
import plotly.graph_objects as go
import plotly.offline as pyo # jupyter notebook 에서 보여지도록 설정하는 부분 (가끔 안나올 때, 이 명령을 하면 됨)
pyo.init_notebook_mode()

fig = go.Figure()
fig.add_trace(go.Scatter(x=predict.index, y=predict[0], name="LSTM 예측", line=dict(color='royalblue', width=1)))
fig.add_trace(go.Scatter(x=original.index, y=original[0], name="실제 주가", line=dict(color='firebrick', width=1)))
fig.update_layout(
    {
        "title": { "text": "딥러닝 주가 예측 (Amazon)", "x": 0.5, "y": 0.9, "font": { "size": 15 } },
        "showlegend": True,
        "xaxis": { "title": "time index" },
        "yaxis": { "title": "price" },
        "template":"ggplot2"
    }
)
fig.show()

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩 (https://school.fun-coding.org/)</a> 에서 본 강의를 포함하는 최적화된 로드맵도 확인하실 수 있습니다</b></div>